Assignment
The buzzer module works as follows
• The ‘-’ pin connects to gnd.
• The ‘+’ pin is the signal you want to write.
• The middle pin is the power (3.3V)
• Writing a square wave (1, 0, 1, 0, 1, 0, etc) alternating high/low, will generate a tone at the
given frequency. Psuedocode looks like this
– while we want a tone
∗ write gpio value high
∗ sleep for 1/(2 * tone_freq)
∗ write gpio_value low
∗ sleep for 1/(2 * tone_freq)




In [1]:
from pynq.overlays.base import BaseOverlay
import time
import asyncio
from datetime import datetime
base = BaseOverlay("base.bit")



In [10]:
%%microblaze base.PMODB

#include "gpio.h"
#include "pyprintf.h"
//#include "stdio.h"

//Function to read the value of a selected pin of PMODB
unsigned int read_gpio(unsigned int pin){
    gpio pin_in = gpio_open(pin);
    gpio_set_direction(pin_in, GPIO_IN);
    return gpio_read(pin_in);
}

//Function to turn on/off a selected pin of PMODB
void write_gpio(unsigned int pin, unsigned int val){
    if (val > 1){
        pyprintf("pin value must be 0 or 1");
    }
    //pyprintf("write to gpio : pin% value%",pin,val);
    //#print("write called");
    //printf("write called");
    read_gpio(pin);
    gpio pin_out = gpio_open(pin);
    gpio_set_direction(pin_out, GPIO_OUT);
    gpio_write(pin_out, val);
    // gpio_close(pin_out);
}


//1. Code to reset all pins
void reset_all_gpio_pins() {
    pyprintf("reset_all_gpio_pins");
     for(int i=0;i<8;i++) {
        write_gpio(i,0);
    }
}

In [11]:
reset_all_gpio_pins();
buzzer=3
cnt=0

write_gpio(1,0)
while cnt<3 :
#while True :
    cnt=cnt+1
    write_gpio(buzzer,0)
    time.sleep(0.5)
    write_gpio(buzzer,1)
    time.sleep(0.5)

write_gpio(buzzer,0)    
print("Done")

Done


In [4]:
reset_all_gpio_pins();

The communication part requires the following
• Using multiprocessing library, create two processes: one process for server and one process for
client.
• Each board will need to have the following happening.
– The server process should always be running in listening mode.
– By pushing one of the buttons on the PYNQ board, the client has to start and connect to
the server board.
– After client connects, pressing a different button should emit a tone on the other PYNQ
board.
∗ Pushing the button should emit a ~0.5 second tone each time it’s pressed.
– By pushing a third button, the client board disconnects from the server. This will end the
communication and both the server and client will terminate
1
• These steps should be completed for each version of PYNQ1 -> PYNQ2. This means pushing a
button on PYNQ1 will emit a tone on PYNQ2 as well as pushing a button on PYNQ2 will emit
a tone on PYNQ1.
– Both directions of communication should be able to operate at the same time.
Deliverables
Each student must submit the following individually
1. A PDF report detailing your work flow and relative Jupyter notebook cells relating to your
progress. * Your report should detail your work flow throughout the assignment. Be sure to
discuss any difficulties or troubles you encountered and your troubleshooting procedure. Also,
detail your thought process which led to the design and implementation of the code. For example,
describe your top-down design methodology (i.e. how did you split the large task into smaller,
more incremental jobs? How were you able to test each of these smaller parts?)
2. Your complete Jupyter notebook, downloaded as a PDF attached at the very end of your
report * You can do this by selecting ‘File -> Print Preview’ then printing to PDF from the
browser. * Use a PDF stitching tool like pdfjoiner to join your Report and Jupyter Notebook
into a single PDF file
3. Video demonstration of your code working on the PYNQ board. Please limit videos to 60 seconds
and upload them to a video sharing site and include the link on your PDF report.
Each team should submit the following one per team
1. All relevant code (.ipynb, .py, .cpp, .c, etc files) pushed to your team’s git repo.

In [5]:
import socket

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# Any iteration of the code below should work
# HOST = '0.0.0.0' 
# HOST = '127.0.0.1'
HOST = ''    # Symbolic name meaning the local host
PORT=1111
start_connection_btn0 = base.buttons[0]
start_buzzer_btn1 = base.buttons[1]
stop_btn2= base.buttons[2]

#IS_THIS_CLIENT=0

In [6]:
def beep_buzzer() :
    reset_all_gpio_pins();
    buzzer=3
    cnt=0

    write_gpio(1,0)
    while cnt<3 :
    #while True :
        cnt=cnt+1
        write_gpio(buzzer,1)
        time.sleep(0.5)
        write_gpio(buzzer,0)
        time.sleep(0.5)

    write_gpio(buzzer,0)    
    print("beep_buzzer Done")

In [68]:
def server_process(HOST,PORT):
    print("in server process")
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind((HOST, PORT))
    s.listen(1)
    conn, addr = s.accept()
    print('Connection from:', addr)

    while True:
        #while True:
        
            data = conn.recv(1024)
            print("Server called.")
            # If there is no data in the connection
            if not data:
            # loop again
                #break
                continue
            # If there is data in the connection
            # Print the data
            print("Received from client: ", data.decode())
            beep_buzzer()
            response = "Beeped Buzzer?"
            conn.sendall(response.encode())

            #if (decoded_code==1) conn.close()

In [69]:
def client_process(HOST,PORT):
    #HOST = '127.0.0.1'  # The server's hostname or IP address
    #PORT = 5000        # The port used by the server
 
    print("on client process")
    got_connection=0
    got_start=0
    while True:
        if (start_connection_btn0.read() != 0 ) or got_connection==1 :
            print("Connection Established")
            got_connection=1
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            s.connect((HOST, PORT))
            print("on client process : After connect")
            while True:
                while True :
                    if (start_buzzer_btn1.read() != 0 ) or got_start==1 :
                        got_start=1
                        message = "How are you?"
                        s.sendall(message.encode())
                        data = s.recv(1024).decode()
                        print("Received from server:", data)
                    else :
                        pass
                        #print ("Button0 NOT PRESED")

                    if stop_btn2.read() != 0 :
                        print("Received STOp request from USER. Break connection", data)
                        #s.sendall(message.encode())
                        #data = s.recv(1024).decode()
                        s.close()
                        break

In [70]:
import multiprocessing
import time
import os


PORT=5003
PORT2=PORT+20
MARIO_IP_ADDR='192.168.0.216'
RAJAN_IP_ADDR='192.168.0.148'

# Pynq 1 : Rajans 
server_process = multiprocessing.Process(target=server_process, args=('',PORT))
#server_process = multiprocessing.Process(target=server_process, args=('',PORT,)) #for self
    

print("Establish server connection")
server_process.start()
# Use taskset command to assign process0 to CPU0
# taskset is an os command to pin the process to a specific CPU
os.system("taskset -pc {} {}".format(0, server_process.pid)) 





Establish server connection
in server process


0

Connection from: ('192.168.0.148', 39058)
Server called.
Received from client:  How are you?
beep_buzzer Done
Server called.
Received from client:  How are you?
beep_buzzer Done
Server called.
Received from client:  How are you?
beep_buzzer Done
Server called.
Received from client:  How are you?
beep_buzzer Done
Server called.
Received from client:  How are you?
beep_buzzer Done
Server called.
Received from client:  How are you?
beep_buzzer Done
Server called.
Server called.
Server called.
Server called.
Server called.
Server called.
Server called.
Server called.
Server called.
Server called.
Server called.
Server called.
Server called.
Server called.
Server called.
Server called.
Server called.
Server called.
Server called.
Server called.
Server called.
Server called.
Server called.
Server called.
Server called.
Server called.
Server called.
Server called.
Server called.
Server called.
Server called.
Server called.
Server called.
Server called.
Server called.
Server called.
Server cal

In [ ]:
#Client Cell
#Connect to Pynq 2 - Marios
print("before client process")
client_process = multiprocessing.Process(target=client_process, args=(RAJAN_IP_ADDR,PORT2)) 

print("start client process")
client_process.start()
print("started client process")
# Use taskset command to assign process0 to CPU0
# taskset is an os command to pin the process to a specific CPU
os.system("taskset -pc {} {}".format(1, client_process.pid)) 
print("client process taskset done")

# Here we wait for client process to finish 
# Then wait for server process to finish
server_process.join()
print("Process 1 with name " + server_process.name + ", is finished")

time.sleep(10)


client_process.join() # wait for process2 to finish
print("Process 1 with name " + client_process.name + ", is finished")

before client process
start client process
on client process
started client process
client process taskset done
Connection Established
on client process : After connect
Received from server: Beeped Buzzer?
Received from server: Beeped Buzzer?
Received from server: Beeped Buzzer?
Received from server: Beeped Buzzer?
Received from server: Beeped Buzzer?
Received from server: Beeped Buzzer?
Received STOp request from USER. Break connection Beeped Buzzer?


Process Process-26:
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-69-23d4f5384f21>", line 20, in client_process
    s.sendall(message.encode())
OSError: [Errno 9] Bad file descriptor


In [13]:
write_gpio(3,0)